## Data load

In [1]:
import torch
import torchvision
from torchvision import models,transforms,datasets

/home/andy/anaconda3/envs/py37/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/andy/anaconda3/envs/py37/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZNK3c1010TensorImpl36is_contiguous_nondefault_policy_implENS_12MemoryFormatE
  warn(f"Failed to load image Python extension: {e}")


In [2]:
import numpy as np

root_dir = './data/MNIST/'
train_set = torchvision.datasets.MNIST(root=root_dir, train=True, download=True)
images = train_set.data.numpy().astype(np.float32)/255
labels = train_set.targets.numpy()
n=len(images)

eights=[images[i] for i in range(n) if labels[i]==8]
ones=[images[i] for i in range(n) if labels[i]==1]

bs = 64

l8 = np.array(0)
eights_dataset = [[torch.from_numpy(e.astype(np.float32)).unsqueeze(0), torch.from_numpy(l8.astype(np.int64))] for e in eights]
l1 = np.array(1)
ones_dataset = [[torch.from_numpy(e.astype(np.float32)).unsqueeze(0), torch.from_numpy(l1.astype(np.int64))] for e in ones]
train_dataset = eights_dataset[1000:] + ones_dataset[1000:]
test_dataset = eights_dataset[:1000] + ones_dataset[:1000]

train_loader = torch.utils.data.DataLoader(train_dataset,
    batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset,
    batch_size=bs, shuffle=True)

In [3]:
import torch.nn as nn
import torch.nn.functional as F
from collections import OrderedDict

class classifier(nn.Module):
    
    def __init__(self):
        super(classifier, self).__init__()
        # fill the missing entries below
        # self.lsfm   = nn.LogSoftmax(dim=1)
        self.feature_layers = nn.Sequential(OrderedDict([
            ('conv1', nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, padding=1)),
            ('pool1', nn.MaxPool2d(kernel_size=2)),
            ('conv2', nn.Conv2d(in_channels=8, out_channels=32, kernel_size=3, padding=1)),
            ('pool2', nn.MaxPool2d(kernel_size=2)),
        ]))
        self.fc = nn.Linear(in_features=32*7*7, out_features=2)
        
    def forward(self,x):
        # implement your network here, use F.max_pool2d, F.log_softmax and do not forget to flatten your vector
        # x = self.conv1(x)
        #
        # Your code here
        #
        #
        # x = F.max_pool2d(x, kernel_size=2)  # (N, C=8, W=14, H=14)
        # x = F.conv2d(x, input_channel=8, output_channel=32, kernel_size=3, padding=1) #(N, C=32, W=14, H=14)
        # x = F.max_pool2d(x, kernel_size=2)  # (N, C=32, W=7, H=7)
        # x = F.linear(x.reshape(x.shape[0], -1), input_features=32*7*7, output_features=128)
        # x = self.fc(x) #(N, 2)
        x = self.feature_layers(x)
        x = self.fc(x.reshape(x.shape[0], -1))
        x = F.log_softmax(x, dim=1) 
        return x

In [4]:
def train(model,data_loader,loss_fn,optimizer,n_epochs=1):
    model.train(True)
    loss_train = np.zeros(n_epochs)
    acc_train = np.zeros(n_epochs)
    for epoch_num in range(n_epochs):
        running_corrects = 0.0
        running_loss = 0.0
        size = 0

        for data in data_loader:
            inputs, labels = data
            bs = labels.size(0)
            #
            #
            # Your code here
            #
            #
            outputs = model(inputs.to(device))
            loss = loss_fn(outputs, labels.to(device))
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            running_loss += loss.cpu()
            preds = torch.argmax(outputs, dim=1)
            acc_es = (preds.cpu() == labels)
            running_corrects = torch.sum(acc_es).cpu()
            size += bs
        epoch_loss = running_loss.item() / size
        epoch_acc = running_corrects.item() / size
        loss_train[epoch_num] = epoch_loss
        acc_train[epoch_num] = epoch_acc
        print('Train - Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))
    return loss_train, acc_train

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [6]:
conv_class = classifier().to(device)

In [7]:

# choose the appropriate loss
loss_fn = nn.NLLLoss()
# your SGD optimizer
learning_rate = 1e-3
optimizer_cl = torch.optim.SGD(conv_class.parameters(), lr=learning_rate)
# and train for 10 epochs
l_t, a_t = train(conv_class,train_loader,loss_fn,optimizer_cl,n_epochs = 10)

Train - Loss: 0.0103 Acc: 0.0026
Train - Loss: 0.0094 Acc: 0.0030
Train - Loss: 0.0085 Acc: 0.0031
Train - Loss: 0.0076 Acc: 0.0030
Train - Loss: 0.0067 Acc: 0.0030
Train - Loss: 0.0058 Acc: 0.0029
Train - Loss: 0.0049 Acc: 0.0029
Train - Loss: 0.0043 Acc: 0.0029
Train - Loss: 0.0037 Acc: 0.0029
Train - Loss: 0.0032 Acc: 0.0029


In [10]:
def test(model,data_loader):
    model.eval()

    running_corrects = 0.0
    running_loss = 0.0
    size = 0

    for data in data_loader:
        inputs, labels = data
            
        bs = labels.size(0)
        #
        # Your code here
        #
        outputs = model(inputs.to(device))
        loss = loss_fn(outputs, labels.to(device))
        
        running_loss += loss.cpu()
        preds = torch.argmax(outputs, dim=1)
        acc_es = (preds.cpu() == labels)
        running_corrects += torch.sum(acc_es).cpu()
            
        size += bs

    print('Test - Loss: {:.4f} Acc: {:.4f}'.format(running_loss / size, running_corrects.item() / size))

In [11]:
test(conv_class, test_loader)

Test - Loss: 0.0033 Acc: 0.9625


In [ ]:
for m in conv_class.children():
    T_w = m.weight.data.numpy()
    T_b = m.bias.data.numpy()
    break